
## BUSINESS PROBLEM: Supply Chain Cost Minimization

- A manufacturing company has 3 factories and needs to supply 4 distribution centers.
Each factory has production capacity constraints, each distribution center has demand requirements,
and there are different transportation costs between each factory-distribution center pair.

### OBJECTIVE: 
- Minimize total transportation costs while meeting all demand and capacity constraints.

- This is a classic Transportation Problem in Operations Research

In [20]:
import pulp as plp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

# Alternative import method if needed
# from pulp import *

# Set up the problem data
print("="*60)
print("SUPPLY CHAIN OPTIMIZATION MODEL")
print("="*60)

# Define factories and distribution centers
factories = ['Factory_A', 'Factory_B', 'Factory_C']
distribution_centers = ['DC_1', 'DC_2', 'DC_3', 'DC_4']

# Supply capacity for each factory (in units)
supply = {
    'Factory_A': 300,
    'Factory_B': 400,
    'Factory_C': 500
}

# Demand for each distribution center (in units)
demand = {
    'DC_1': 200,
    'DC_2': 300,
    'DC_3': 400,
    'DC_4': 300
}

# Transportation cost matrix (cost per unit from factory i to DC j)
cost_matrix = {
    'Factory_A': {'DC_1': 8, 'DC_2': 6, 'DC_3': 10, 'DC_4': 9},
    'Factory_B': {'DC_1': 9, 'DC_2': 12, 'DC_3': 13, 'DC_4': 7},
    'Factory_C': {'DC_1': 14, 'DC_2': 9, 'DC_3': 16, 'DC_4': 5}
}

print("\nPROBLEM SETUP:")
print("-" * 30)
print(f"Factories: {factories}")
print(f"Distribution Centers: {distribution_centers}")
print(f"Total Supply: {sum(supply.values())} units")
print(f"Total Demand: {sum(demand.values())} units")

# Display the cost matrix
print("\nTransportation Cost Matrix ($/unit):")
cost_df = pd.DataFrame(cost_matrix).T
print(cost_df)

# Check if the problem is balanced (supply = demand)
total_supply = sum(supply.values())
total_demand = sum(demand.values())
print(f"\nProblem Balance Check:")
print(f"Total Supply: {total_supply}")
print(f"Total Demand: {total_demand}")
print(f"Balanced: {'Yes' if total_supply == total_demand else 'No'}")

print("\n" + "="*60)
print("OPTIMIZATION MODEL FORMULATION")
print("="*60)

# Create the linear programming problem
prob = plp.LpProblem("Supply_Chain_Optimization", plp.LpMinimize)

# Create decision variables
# x[i,j] = quantity shipped from factory i to distribution center j

SUPPLY CHAIN OPTIMIZATION MODEL

PROBLEM SETUP:
------------------------------
Factories: ['Factory_A', 'Factory_B', 'Factory_C']
Distribution Centers: ['DC_1', 'DC_2', 'DC_3', 'DC_4']
Total Supply: 1200 units
Total Demand: 1200 units

Transportation Cost Matrix ($/unit):
           DC_1  DC_2  DC_3  DC_4
Factory_A     8     6    10     9
Factory_B     9    12    13     7
Factory_C    14     9    16     5

Problem Balance Check:
Total Supply: 1200
Total Demand: 1200
Balanced: Yes

OPTIMIZATION MODEL FORMULATION


In [18]:
x = {}
for i in factories:
    for j in distribution_centers:
        x[i, j] = plp.LpVariable(f"x_{i}_{j}", lowBound=0, cat='Integer')

print("Decision Variables:")
print("x[i,j] = quantity shipped from factory i to distribution center j")
print("where i ∈ {Factory_A, Factory_B, Factory_C}")
print("      j ∈ {DC_1, DC_2, DC_3, DC_4}")

# Objective function: minimize total transportation cost
total_cost = plp.lpSum([cost_matrix[i][j] * x[i, j] 
                        for i in factories for j in distribution_centers])
prob += total_cost

print("\nObjective Function:")
print("Minimize: Σ(cost[i,j] * x[i,j]) for all i,j")

# Supply constraints: each factory cannot exceed its capacity
print("\nSupply Constraints:")
for i in factories:
    prob += plp.lpSum([x[i, j] for j in distribution_centers]) <= supply[i]
    print(f"Σ x[{i},j] ≤ {supply[i]} for all j")

# Demand constraints: each distribution center must receive its required demand
print("\nDemand Constraints:")
for j in distribution_centers:
    prob += plp.lpSum([x[i, j] for i in factories]) >= demand[j]
    print(f"Σ x[i,{j}] ≥ {demand[j]} for all i")

print("\nNon-negativity Constraints:")
print("x[i,j] ≥ 0 for all i,j")

print("\n" + "="*60)
print("SOLVING THE OPTIMIZATION PROBLEM")
print("="*60)

# Solve the problem
prob.solve()

# Check the solution status
print(f"Status: {plp.LpStatus[prob.status]}")

if prob.status == plp.LpStatusOptimal:
    print("\nOPTIMAL SOLUTION FOUND!")
    print("-" * 30)
    
    # Extract the solution
    solution = {}
    for i in factories:
        solution[i] = {}
        for j in distribution_centers:
            solution[i][j] = x[i, j].varValue
    
    # Display the solution
    print("Optimal Shipping Quantities:")
    solution_df = pd.DataFrame(solution).T
    print(solution_df)
    
    # Calculate total cost
    optimal_cost = plp.value(prob.objective)
    print(f"\nMinimum Total Transportation Cost: ${optimal_cost:,.2f}")
    
    # Verify constraints are satisfied
    print("\nCONSTRAINT VERIFICATION:")
    print("-" * 30)
    
    # Check supply constraints
    print("Supply Utilization:")
    for i in factories:
        used_capacity = sum(solution[i][j] for j in distribution_centers)
        print(f"{i}: {used_capacity}/{supply[i]} units ({used_capacity/supply[i]*100:.1f}%)")
    
    # Check demand satisfaction
    print("\nDemand Satisfaction:")
    for j in distribution_centers:
        received = sum(solution[i][j] for i in factories)
        print(f"{j}: {received}/{demand[j]} units ({received/demand[j]*100:.1f}%)")
    
    print("\n" + "="*60)
    print("SOLUTION ANALYSIS & INSIGHTS")
    print("="*60)
    
    # INSIGHT 1: Supply Chain Network Analysis
    print("🔍 KEY INSIGHTS:")
    print("-" * 50)
    
    # Network complexity analysis
    total_possible_routes = len(factories) * len(distribution_centers)
    active_routes_count = sum(1 for i in factories for j in distribution_centers if solution[i][j] > 0)
    network_utilization = active_routes_count / total_possible_routes
    
    print(f"📊 NETWORK EFFICIENCY:")
    print(f"   • Active routes: {active_routes_count}/{total_possible_routes} ({network_utilization:.1%})")
    
    if network_utilization < 0.5:
        print("   ✅ INSIGHT: Lean network - Cost-efficient with minimal route complexity")
    elif network_utilization > 0.8:
        print("   ⚠️  INSIGHT: Dense network - May indicate inefficient routing or balanced costs")
    else:
        print("   ✅ INSIGHT: Balanced network - Good mix of efficiency and flexibility")
    
    # INSIGHT 2: Cost Structure Analysis
    print(f"\n💰 COST STRUCTURE ANALYSIS:")
    total_volume = sum(demand.values())
    avg_distance_cost = optimal_cost / total_volume
    
    # Find cost variance across routes
    active_costs = [cost_matrix[i][j] for i in factories for j in distribution_centers if solution[i][j] > 0]
    cost_variance = np.var(active_costs) if active_costs else 0
    cost_std = np.std(active_costs) if active_costs else 0
    
    print(f"   • Average cost per unit: ${avg_distance_cost:.2f}")
    print(f"   • Cost variance across routes: {cost_variance:.2f}")
    print(f"   • Cost standard deviation: ${cost_std:.2f}")
    
    if cost_std < 2:
        print("   ✅ INSIGHT: Low cost variation - Uniform transportation costs")
    elif cost_std > 5:
        print("   ⚠️  INSIGHT: High cost variation - Significant geographic or logistical differences")
    else:
        print("   ✅ INSIGHT: Moderate cost variation - Normal market conditions")
    
    # INSIGHT 3: Capacity Utilization Patterns
    print(f"\n🏭 CAPACITY UTILIZATION INSIGHTS:")
    utilization_rates = []
    for i in factories:
        used = sum(solution[i][j] for j in distribution_centers)
        utilization = used / supply[i]
        utilization_rates.append(utilization)
        
    avg_utilization = np.mean(utilization_rates)
    utilization_balance = 1 - np.std(utilization_rates)  # Higher = more balanced
    
    print(f"   • Average capacity utilization: {avg_utilization:.1%}")
    print(f"   • Utilization balance score: {utilization_balance:.3f}")
    
    if avg_utilization > 0.85:
        print("   ⚠️  INSIGHT: High utilization - Limited flexibility for demand spikes")
        print("   💡 RECOMMENDATION: Consider capacity expansion or demand smoothing")
    elif avg_utilization < 0.65:
        print("   ✅ INSIGHT: Conservative utilization - Good buffer for demand fluctuations")
        print("   💡 OPPORTUNITY: Potential for cost reduction through capacity optimization")
    else:
        print("   ✅ INSIGHT: Optimal utilization - Good balance of efficiency and flexibility")
    
    if utilization_balance > 0.8:
        print("   ✅ INSIGHT: Well-balanced factory utilization across network")
    else:
        print("   ⚠️  INSIGHT: Unbalanced utilization - Some factories over/under-utilized")
    
    # INSIGHT 4: Demand Fulfillment Strategy
    print(f"\n🎯 DEMAND FULFILLMENT STRATEGY:")
    
    # Analyze supplier diversity for each DC
    supplier_diversity = {}
    for j in distribution_centers:
        suppliers = [i for i in factories if solution[i][j] > 0]
        supplier_diversity[j] = len(suppliers)
    
    avg_suppliers_per_dc = np.mean(list(supplier_diversity.values()))
    print(f"   • Average suppliers per DC: {avg_suppliers_per_dc:.1f}")
    
    # Risk assessment
    single_supplier_dcs = [dc for dc, count in supplier_diversity.items() if count == 1]
    if single_supplier_dcs:
        print(f"   ⚠️  RISK: {len(single_supplier_dcs)} DCs depend on single suppliers")
        print(f"   💡 MITIGATION: Diversify supply sources for: {', '.join(single_supplier_dcs)}")
    else:
        print("   ✅ INSIGHT: Good supplier diversification - Reduced supply chain risk")
    
    # INSIGHT 5: Economic Efficiency Metrics
    print(f"\n📈 ECONOMIC EFFICIENCY METRICS:")
    
    # Calculate theoretical minimum cost (if each DC got from cheapest supplier)
    theoretical_min_cost = 0
    for j in distribution_centers:
        min_cost = min(cost_matrix[i][j] for i in factories)
        theoretical_min_cost += min_cost * demand[j]
    
    efficiency_ratio = theoretical_min_cost / optimal_cost
    efficiency_gap = optimal_cost - theoretical_min_cost
    
    print(f"   • Theoretical minimum cost: ${theoretical_min_cost:,.2f}")
    print(f"   • Actual optimal cost: ${optimal_cost:,.2f}")
    print(f"   • Efficiency ratio: {efficiency_ratio:.3f}")
    print(f"   • Efficiency gap: ${efficiency_gap:,.2f}")
    
    if efficiency_ratio > 0.90:
        print("   ✅ INSIGHT: Highly efficient solution - Near-optimal cost structure")
    elif efficiency_ratio > 0.80:
        print("   ✅ INSIGHT: Good efficiency - Reasonable balance of cost and constraints")
    else:
        print("   ⚠️  INSIGHT: Efficiency opportunity - Significant cost gap due to constraints")
    
    # INSIGHT 6: Operational Complexity Score
    print(f"\n⚙️ OPERATIONAL COMPLEXITY ANALYSIS:")
    
    # Calculate complexity metrics
    route_complexity = active_routes_count / len(factories)  # Routes per factory
    volume_complexity = np.std([sum(solution[i][j] for j in distribution_centers) for i in factories])
    
    print(f"   • Average routes per factory: {route_complexity:.1f}")
    print(f"   • Volume distribution complexity: {volume_complexity:.1f}")
    
    if route_complexity < 2:
        print("   ✅ INSIGHT: Simple routing - Easy to manage and execute")
    elif route_complexity > 3:
        print("   ⚠️  INSIGHT: Complex routing - May require advanced logistics coordination")
    else:
        print("   ✅ INSIGHT: Moderate complexity - Manageable with standard operations")
    
    # INSIGHT 7: Market Position Analysis
    print(f"\n🌍 MARKET POSITION INSIGHTS:")
    
    # Calculate market coverage efficiency
    factory_workload = {i: sum(solution[i][j] for j in distribution_centers) for i in factories}
    dominant_factory = max(factory_workload, key=factory_workload.get)
    market_dominance = factory_workload[dominant_factory] / sum(factory_workload.values())
    
    print(f"   • Dominant factory: {dominant_factory} ({market_dominance:.1%} of total volume)")
    
    if market_dominance > 0.6:
        print("   ⚠️  INSIGHT: High concentration risk - Over-dependence on single facility")
        print("   💡 STRATEGY: Develop backup capacity or improve load balancing")
    elif market_dominance < 0.4:
        print("   ✅ INSIGHT: Well-distributed load - Good risk management")
    else:
        print("   ✅ INSIGHT: Balanced market position - Reasonable load distribution")
    
    # INSIGHT 8: Scalability Assessment
    print(f"\n🚀 SCALABILITY INSIGHTS:")
    
    spare_capacity = sum(supply.values()) - sum(demand.values())
    scalability_buffer = spare_capacity / sum(demand.values())
    
    print(f"   • Spare capacity: {spare_capacity} units")
    print(f"   • Scalability buffer: {scalability_buffer:.1%}")
    
    if scalability_buffer > 0.2:
        print("   ✅ INSIGHT: High scalability - Can handle significant demand growth")
    elif scalability_buffer > 0.1:
        print("   ✅ INSIGHT: Moderate scalability - Can accommodate normal growth")
    else:
        print("   ⚠️  INSIGHT: Limited scalability - Near capacity limits")
        print("   💡 PLANNING: Consider capacity expansion for future growth")
    
    # Create visualization data
    # 1. Cost breakdown by factory
    factory_costs = {}
    for i in factories:
        factory_costs[i] = sum(cost_matrix[i][j] * solution[i][j] for j in distribution_centers)
    
    print("Cost Breakdown by Factory:")
    for factory, cost in factory_costs.items():
        print(f"{factory}: ${cost:,.2f} ({cost/optimal_cost*100:.1f}%)")
    
    # 2. Route analysis
    print("\nActive Shipping Routes (Quantity > 0):")
    active_routes = []
    for i in factories:
        for j in distribution_centers:
            if solution[i][j] > 0:
                route_cost = cost_matrix[i][j] * solution[i][j]
                active_routes.append({
                    'Route': f"{i} → {j}",
                    'Quantity': solution[i][j],
                    'Unit_Cost': cost_matrix[i][j],
                    'Total_Cost': route_cost
                })
    
    routes_df = pd.DataFrame(active_routes)
    print(routes_df.to_string(index=False))
    
    # 3. Efficiency metrics
    print("\nEFFICIENCY METRICS:")
    print("-" * 20)
    
    # Average cost per unit
    total_units = sum(demand.values())
    avg_cost_per_unit = optimal_cost / total_units
    print(f"Average Transportation Cost per Unit: ${avg_cost_per_unit:.2f}")
    
    # Capacity utilization
    total_capacity = sum(supply.values())
    capacity_utilization = total_units / total_capacity * 100
    print(f"Overall Capacity Utilization: {capacity_utilization:.1f}%")
    
    # Number of active routes
    total_possible_routes = len(factories) * len(distribution_centers)
    active_route_count = len(active_routes)
    print(f"Active Routes: {active_route_count}/{total_possible_routes}")
    
    print("\n" + "="*60)
    print("SENSITIVITY ANALYSIS")
    print("="*60)
    
    # Analyze what happens if demand increases by 10%
    print("Scenario Analysis: 10% Demand Increase")
    print("-" * 40)
    
    # New demand scenario
    new_demand = {k: int(v * 1.1) for k, v in demand.items()}
    new_total_demand = sum(new_demand.values())
    
    print(f"Original Total Demand: {total_demand} units")
    print(f"New Total Demand: {new_total_demand} units")
    print(f"Total Available Supply: {total_supply} units")
    
    if new_total_demand <= total_supply:
        print("✓ Increased demand can be satisfied with current capacity")
        
        # Solve with new demand
        prob_scenario = plp.LpProblem("Supply_Chain_Scenario", plp.LpMinimize)
        
        # Same variables and objective
        x_scenario = {}
        for i in factories:
            for j in distribution_centers:
                x_scenario[i, j] = plp.LpVariable(f"x_scenario_{i}_{j}", lowBound=0, cat='Integer')
        
        total_cost_scenario = plp.lpSum([cost_matrix[i][j] * x_scenario[i, j] 
                                        for i in factories for j in distribution_centers])
        prob_scenario += total_cost_scenario
        
        # Same supply constraints
        for i in factories:
            prob_scenario += plp.lpSum([x_scenario[i, j] for j in distribution_centers]) <= supply[i]
        
        # New demand constraints
        for j in distribution_centers:
            prob_scenario += plp.lpSum([x_scenario[i, j] for i in factories]) >= new_demand[j]
        
        prob_scenario.solve()
        
        if prob_scenario.status == plp.LpStatusOptimal:
            new_optimal_cost = plp.value(prob_scenario.objective)
            cost_increase = new_optimal_cost - optimal_cost
            print(f"New Optimal Cost: ${new_optimal_cost:,.2f}")
            print(f"Cost Increase: ${cost_increase:,.2f} ({cost_increase/optimal_cost*100:.1f}%)")
        
    else:
        capacity_deficit = new_total_demand - total_supply
        print(f"✗ Demand exceeds capacity by {capacity_deficit} units")
        print("  Consider expanding production capacity or adding suppliers")
    
    print("\n" + "="*60)
    print("STRATEGIC BUSINESS RECOMMENDATIONS")
    print("="*60)
    
    print("🎯 IMMEDIATE ACTIONS:")
    print("-" * 30)
    
    # Generate dynamic recommendations based on insights
    recommendations = []
    
    # Cost optimization recommendations
    expensive_routes = sorted([(i, j, cost_matrix[i][j], solution[i][j]) 
                              for i in factories for j in distribution_centers 
                              if solution[i][j] > 0], key=lambda x: x[2], reverse=True)
    
    if expensive_routes:
        top_expensive = expensive_routes[:3]
        print("1. COST REDUCTION PRIORITIES:")
        for i, (factory, dc, cost, volume) in enumerate(top_expensive, 1):
            total_route_cost = cost * volume
            print(f"   {i}. {factory} → {dc}: ${cost}/unit × {volume} units = ${total_route_cost:,.2f}")
        
        # Find alternative routes
        print("\n   💡 ALTERNATIVE ROUTE ANALYSIS:")
        for factory, dc, cost, volume in expensive_routes[:2]:
            alternatives = [(f, cost_matrix[f][dc]) for f in factories if f != factory 
                           and sum(solution[f][j] for j in distribution_centers) < supply[f]]
            if alternatives:
                best_alt = min(alternatives, key=lambda x: x[1])
                savings = (cost - best_alt[1]) * volume
                print(f"   • {factory} → {dc}: Switch to {best_alt[0]} saves ${savings:,.2f}")
    
    # Capacity optimization
    print("\n2. CAPACITY OPTIMIZATION:")
    underutilized = [(i, sum(solution[i][j] for j in distribution_centers)/supply[i]) 
                     for i in factories]
    underutilized.sort(key=lambda x: x[1])
    
    if underutilized[0][1] < 0.8:
        print(f"   • {underutilized[0][0]} is underutilized ({underutilized[0][1]:.1%})")
        print(f"   💡 Consider: Reduce capacity or reallocate production")
    
    # Risk mitigation
    print("\n3. RISK MITIGATION:")
    vulnerable_dcs = [dc for dc, suppliers in supplier_diversity.items() if suppliers == 1]
    if vulnerable_dcs:
        print(f"   • High-risk DCs (single supplier): {', '.join(vulnerable_dcs)}")
        print("   💡 Action: Develop secondary supply routes")
    
    # Market opportunities
    print("\n4. MARKET OPPORTUNITIES:")
    if scalability_buffer > 0.15:
        print(f"   • {scalability_buffer:.1%} spare capacity available")
        print("   💡 Opportunity: Pursue additional customers or markets")
    
    # Operational improvements
    print("\n5. OPERATIONAL IMPROVEMENTS:")
    if network_utilization < 0.6:
        print("   • Streamlined network with efficient routing")
        print("   💡 Focus: Maintain operational excellence")
    else:
        print("   • Complex network may benefit from consolidation")
        print("   💡 Consider: Route optimization or facility consolidation")
    
    print("\n" + "="*40)
    print("📊 PERFORMANCE BENCHMARKS")
    print("="*40)
    
    # Create performance dashboard
    performance_metrics = {
        'Cost Efficiency': f"{efficiency_ratio:.3f}",
        'Network Utilization': f"{network_utilization:.1%}",
        'Capacity Utilization': f"{avg_utilization:.1%}",
        'Supplier Diversity': f"{avg_suppliers_per_dc:.1f}",
        'Scalability Buffer': f"{scalability_buffer:.1%}",
        'Operational Complexity': f"{route_complexity:.1f}"
    }
    
    print("Key Performance Indicators:")
    for metric, value in performance_metrics.items():
        print(f"   • {metric}: {value}")
    
    # Scoring system
    print("\n📈 OVERALL SUPPLY CHAIN SCORE:")
    score_components = {
        'Cost Efficiency': min(efficiency_ratio * 100, 100),
        'Risk Management': min((1 - len(vulnerable_dcs)/len(distribution_centers)) * 100, 100),
        'Capacity Utilization': min(avg_utilization * 100, 100),
        'Operational Simplicity': min((1 - route_complexity/4) * 100, 100)
    }
    
    overall_score = sum(score_components.values()) / len(score_components)
    
    for component, score in score_components.items():
        print(f"   • {component}: {score:.1f}/100")
    
    print(f"\n🏆 OVERALL SCORE: {overall_score:.1f}/100")
    
    if overall_score >= 80:
        print("   ✅ EXCELLENT: World-class supply chain performance")
    elif overall_score >= 70:
        print("   ✅ GOOD: Above-average performance with minor optimization opportunities")
    elif overall_score >= 60:
        print("   ⚠️  FAIR: Average performance with clear improvement areas")
    else:
        print("   ❌ NEEDS IMPROVEMENT: Significant optimization required")
    
    print("\n" + "="*60)
    print("SCENARIO PLANNING & FUTURE CONSIDERATIONS")
    print("="*60)
    
    print("🔮 WHAT-IF SCENARIOS:")
    print("-" * 25)
    
    # Scenario 1: Demand surge
    print("1. DEMAND SURGE SCENARIO (+20%):")
    surge_demand = {k: int(v * 1.2) for k, v in demand.items()}
    surge_total = sum(surge_demand.values())
    if surge_total <= sum(supply.values()):
        print(f"   ✅ Can handle {surge_total} units (current capacity: {sum(supply.values())})")
        print(f"   💡 Estimated cost increase: ~{(surge_total/sum(demand.values()) - 1)*100:.1f}%")
    else:
        deficit = surge_total - sum(supply.values())
        print(f"   ❌ Capacity deficit: {deficit} units")
        print(f"   💡 Need additional capacity or external suppliers")
    
    # Scenario 2: Factory disruption
    print("\n2. FACTORY DISRUPTION SCENARIO:")
    critical_factory = max(factory_workload, key=factory_workload.get)
    remaining_capacity = sum(supply[f] for f in factories if f != critical_factory)
    current_demand = sum(demand.values())
    
    if remaining_capacity >= current_demand:
        print(f"   ✅ Can survive {critical_factory} disruption")
        print(f"   💡 Remaining capacity: {remaining_capacity} vs demand: {current_demand}")
    else:
        shortfall = current_demand - remaining_capacity
        print(f"   ❌ Cannot meet demand without {critical_factory}")
        print(f"   💡 Shortfall: {shortfall} units ({shortfall/current_demand:.1%})")
    
    # Scenario 3: Cost inflation
    print("\n3. TRANSPORTATION COST INFLATION (+15%):")
    inflated_cost = optimal_cost * 1.15
    cost_impact = inflated_cost - optimal_cost
    print(f"   💰 New total cost: ${inflated_cost:,.2f}")
    print(f"   📈 Additional cost burden: ${cost_impact:,.2f}")
    print(f"   💡 Consider: Long-term contracts or alternative transportation modes")
    
    print("\n🔄 CONTINUOUS IMPROVEMENT ROADMAP:")
    print("-" * 40)
    
    improvement_areas = []
    
    if efficiency_ratio < 0.85:
        improvement_areas.append("Cost optimization through route refinement")
    if avg_utilization < 0.75:
        improvement_areas.append("Capacity utilization improvement")
    if len(vulnerable_dcs) > 0:
        improvement_areas.append("Supply chain risk reduction")
    if route_complexity > 2.5:
        improvement_areas.append("Operational simplification")
    
    for i, area in enumerate(improvement_areas, 1):
        print(f"   {i}. {area}")
    
    if not improvement_areas:
        print("   ✅ Supply chain is well-optimized!")
        print("   💡 Focus on: Maintaining performance and monitoring changes")
    
    print("\n📋 IMPLEMENTATION TIMELINE:")
    print("   • Month 1: Implement immediate cost reduction measures")
    print("   • Month 2-3: Address capacity and risk issues")
    print("   • Month 4-6: Optimize operations and processes")
    print("   • Month 7-12: Strategic improvements and expansion planning")
    
else:
    print("No optimal solution found. Check problem constraints and data.")

print("\n" + "="*60)
print("OPTIMIZATION MODEL SUMMARY")
print("="*60)
print("✓ Problem formulated as Linear Programming model")
print("✓ Optimal solution found using PuLP solver")
print("✓ All constraints satisfied")
print("✓ Comprehensive analysis and insights provided")
print("✓ Business recommendations generated")
print("="*60)

Decision Variables:
x[i,j] = quantity shipped from factory i to distribution center j
where i ∈ {Factory_A, Factory_B, Factory_C}
      j ∈ {DC_1, DC_2, DC_3, DC_4}

Objective Function:
Minimize: Σ(cost[i,j] * x[i,j]) for all i,j

Supply Constraints:
Σ x[Factory_A,j] ≤ 300 for all j
Σ x[Factory_B,j] ≤ 400 for all j
Σ x[Factory_C,j] ≤ 500 for all j

Demand Constraints:
Σ x[i,DC_1] ≥ 200 for all i
Σ x[i,DC_2] ≥ 300 for all i
Σ x[i,DC_3] ≥ 400 for all i
Σ x[i,DC_4] ≥ 300 for all i

Non-negativity Constraints:
x[i,j] ≥ 0 for all i,j

SOLVING THE OPTIMIZATION PROBLEM
Status: Optimal

OPTIMAL SOLUTION FOUND!
------------------------------
Optimal Shipping Quantities:
            DC_1   DC_2   DC_3   DC_4
Factory_A    0.0  100.0  200.0    0.0
Factory_B  200.0    0.0  200.0    0.0
Factory_C    0.0  200.0    0.0  300.0

Minimum Total Transportation Cost: $10,300.00

CONSTRAINT VERIFICATION:
------------------------------
Supply Utilization:
Factory_A: 300.0/300 units (100.0%)
Factory_B: 400.0/4